# TCX Reader - Training Center XML Format

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import time
import re

from lxml import etree
from dateutil.parser import parse

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
activityExtensionNs = 'http://www.garmin.com/xmlschemas/ActivityExtension/'

class TcxReader(BaseReader):
    '''TCX file - Training Center XML Format'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self):
        '''Load file into memory'''

        # Parse TCX file using lxml
        tree = etree.parse(self.filename)
        root = tree.getroot()
        
        # Create namespace mappings that include default
        nsmap = {ns if ns is not None else 'default': url for ns, url in root.nsmap.items()}
        defaultNs = nsmap['default']
        
        # Will use regular expressions to handle namespaces
        pattern = re.compile('{(.*)}(.*)')

        # Iterate through all tracks
        activities = root.xpath(".//default:Activity", namespaces=nsmap)       
        for activity in activities:

            # Create new track object
            track = self.addTrack()

            # Find all trackpoints
            trkpts = activity.xpath(".//default:Trackpoint", namespaces=nsmap)
            maxRecords = len(trkpts)
            numRecords = 0

            # Create empty ndarrays
            ts = np.zeros(maxRecords, dtype='float64')
            lat = np.zeros(maxRecords, dtype='float64')
            lon = np.zeros(maxRecords, dtype='float64')
            ele = np.zeros(maxRecords, dtype='float64')
            hr = np.zeros(maxRecords, dtype='uint16')
            sog = np.zeros(maxRecords, dtype='float32')

            # Process individual trackpoints
            for i in range(maxRecords):
                # Start by determining if latitude and longitude are present
                positions = trkpts[i].xpath(".//default:Position", namespaces=nsmap)
                if len(positions) > 0:
                    for element in positions[0].iter():
                        elementNs, elementName =  pattern.findall(element.tag)[0]
                        if elementName == 'LatitudeDegrees':
                            lat[numRecords] = element.text
                        elif elementName == 'LongitudeDegrees':
                            lon[numRecords] = element.text

                    # Since latitude and longitude are present, look for other elements within the trackpoint
                    for element in trkpts[i].getchildren():
                        elementNs, elementName =  pattern.findall(element.tag)[0]
                        if elementNs == defaultNs and element.text is not None:

                            # Standard TCX elements 
                            if elementName == 'Time':
                                ts[numRecords] = parse(element.text).timestamp()
                            elif elementName == 'AltitudeMeters':
                                ele[numRecords] = element.text
                            elif elementName == 'HeartRateBpm':
                                for element in element.iter():
                                    elementNs, elementName =  pattern.findall(element.tag)[0]
                                    if elementName == 'Value':
                                        hr[numRecords] = element.text

                            # Speed is within an extension, much like in GPX 1.1
                            elif elementName == 'Extensions':
                                for element in element.iter():
                                    if element.text is not None:
                                        elementNs, elementName =  pattern.findall(element.tag)[0]

                                        # Garmin ActivityExtension - either v1 or v2
                                        if elementNs.startswith(activityExtensionNs):
                                            if elementName == 'Speed':
                                                sog[numRecords] = element.text

                                        # COROS - incorrect use of extensions
                                        elif elementNs == defaultNs:
                                            if elementName == 'Speed':
                                                sog[numRecords] = element.text

                    # Ensure trackpoints are not duplicated, due to implementation of laps, etc
                    if numRecords == 0 or ts[numRecords] > ts[numRecords - 1]:
                        numRecords += 1

            # Retain populated arrays
            if np.count_nonzero(ts) > 0:
                track.data['ts'] = np.round(ts[:numRecords], 3)
            if np.count_nonzero(lat) > 0:
                track.data['lat'] = np.round(lat[:numRecords], 7)
            if np.count_nonzero(lon) > 0:
                track.data['lon'] = np.round(lon[:numRecords], 7)
            if np.count_nonzero(ele) > 0:
                track.data['ele'] = np.round(ele[:numRecords], 3)
            if np.count_nonzero(hr) > 0:
                track.data['hr'] = hr[:numRecords]
            if np.count_nonzero(sog) > 0:
                track.data['sog'] = np.round(sog[:numRecords], 3)

        self.summarise()

## Unit Tests

In [3]:
class TestGarminConnectData(unittest.TestCase):
    '''Class to test Garmin Connect data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        # numTrkPts - (numLaps - 1) - empty = 2330 - (24 - 1) - 20 = 2287
        for fieldName in garminConnectReader.tracks[0].data:
            self.assertEqual(garminConnectReader.tracks[0].data[fieldName].size, 2287)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(garminConnectReader.tracks[0].data['ts'].min(), np.float64(1654950221.0))
        self.assertEqual(garminConnectReader.tracks[0].data['ts'].max(), np.float64(1654955409.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(garminConnectReader.tracks[0].data['lat'].min(), np.float64(50.5723549))
        self.assertEqual(garminConnectReader.tracks[0].data['lat'].max(), np.float64(50.5821052))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(garminConnectReader.tracks[0].data['lon'].min(), np.float64(-2.4688928))
        self.assertEqual(garminConnectReader.tracks[0].data['lon'].max(), np.float64(-2.4560265))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(garminConnectReader.tracks[0].data['ele'].min(), np.float64(-11.0))
        self.assertEqual(garminConnectReader.tracks[0].data['ele'].max(), np.float64(7.8))


    def testHr(self):
        '''Test the heart rate is as expected'''

        self.assertEqual(garminConnectReader.tracks[0].data['hr'].min(), np.uint16(82))
        self.assertEqual(garminConnectReader.tracks[0].data['hr'].max(), np.uint16(134))


    def testSog(self):
        '''Test the SOG is as expected'''

        # Real minimum is 0.457
        self.assertEqual(garminConnectReader.tracks[0].data['sog'].min(), np.float32(0))
        self.assertEqual(garminConnectReader.tracks[0].data['sog'].max(), np.float32(13.968))

In [4]:
class TestCoros(unittest.TestCase):
    '''Class to test Coros data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        # numTrkPts - (numLaps - 1) - empty = 11701 - (15 - 1) - 3393 = 8294
        for fieldName in corosReader.tracks[0].data:
            self.assertEqual(corosReader.tracks[0].data[fieldName].size, 8294)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        # Timestamps from 1668256416 to 1668268102 are present, but lat + lon are missing at start and end
        self.assertEqual(corosReader.tracks[0].data['ts'].min(), np.float64(1668256433.0))
        self.assertEqual(corosReader.tracks[0].data['ts'].max(), np.float64(1668268098.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(corosReader.tracks[0].data['lat'].min(), np.float64(50.5953173))
        self.assertEqual(corosReader.tracks[0].data['lat'].max(), np.float64(50.6113483))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(corosReader.tracks[0].data['lon'].min(), np.float64(-2.1307967))
        self.assertEqual(corosReader.tracks[0].data['lon'].max(), np.float64(-2.0816625))


    def testEle(self):
        '''Test the elevation is as expected'''

        # Altitude of -9 is present in TCX, but only when latitude and longitude is missing
        self.assertEqual(corosReader.tracks[0].data['ele'].min(), np.float64(-8))
        self.assertEqual(corosReader.tracks[0].data['ele'].max(), np.float64(81))


    def testHr(self):
        '''Test the heart rate is as expected'''

        # Heartrate of 184 is present in TCX, but only when latitude and longitude is missing
        self.assertEqual(corosReader.tracks[0].data['hr'].min(), np.uint16(75))
        self.assertEqual(corosReader.tracks[0].data['hr'].max(), np.uint16(182))


    def testSog(self):
        '''Test the SOG is as expected'''

        # Real minimum is 1.16
        self.assertEqual(corosReader.tracks[0].data['sog'].min(), np.float32(0))
        self.assertEqual(corosReader.tracks[0].data['sog'].max(), np.float32(11.52))

In [5]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220611', 'activity_8998087955.tcx')
    garminConnectReader = TcxReader(filename)

    filename = os.path.join(projdir, 'sessions', '20221112', 'Walk20221112123336.tcx')
    corosReader = TcxReader(filename)

    pc1 = time.perf_counter()
    garminConnectReader.load()
    corosReader.load()
    pc2 = time.perf_counter()

    print("\nTCX files loaded in in %0.2f seconds" % (pc2 - pc1))


TCX files loaded in in 1.10 seconds


In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..............
----------------------------------------------------------------------
Ran 14 tests in 0.016s

OK
